<a href="https://colab.research.google.com/github/ntxuan1799/Aboriginal-Project/blob/main/Cleaned_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [43]:
df = pd.read_csv("merged_socialmedia_data.csv")

In [44]:
# Initial cleaning
df = df[df['Content'].apply(lambda x: isinstance(x, str))]
df['Content'] = df['Content'].str.lower()
df['Content'] = df['Content'].str.encode('ascii', 'ignore').str.decode('ascii')
df['Title'] = df['Title'].str.encode('ascii', 'ignore').str.decode('ascii')

def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"@\w+|#\w+", '', text)
    text = re.sub(r"\d+", '', text)
    return text

df['Content'] = df['Content'].apply(clean_text)
df['Content'] = df['Content'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [45]:
def parse_mixed_date(val):
    try:
        # Handle missing or NaN
        if pd.isna(val) or val in ["N/A", ""]:
            return pd.Timestamp("2023-10-14").date()

        val_str = str(val).strip()

        # If it's a number and looks like a Unix timestamp
        if val_str.isdigit():
            ts_int = int(val_str)
            if 1000000000 < ts_int < 2000000000:  # UNIX timestamp range
                return pd.to_datetime(ts_int, unit='s').date()

        # Handle ISO 8601 format (e.g., 2023-09-07T00:14:31Z)
        if 'T' in val_str:
            return pd.to_datetime(val_str, utc=True).date()

        # Handle string date format (e.g., 30/1/2025 or 20/6/2024 14:31)
        return pd.to_datetime(val_str, dayfirst=True, errors='coerce').date()

    except:
        return pd.Timestamp("2023-10-14").date()

# Apply conversion
df['Date'] = df['Date'].apply(parse_mixed_date)

# Confirm format
df['Date'] = pd.to_datetime(df['Date']).dt.date

print("All Date values are now in format YYYY-MM-DD with missing values filled as 2023-10-14.")

All Date values are now in format YYYY-MM-DD with missing values filled as 2023-10-14.


In [46]:
# Convert Date column to datetime and remove timezone information
df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.tz_localize(None)
df['day'] = df['Date'].dt.date

referendum_date = pd.Timestamp("2023-10-14").date()
df['Period'] = df['day'].apply(lambda x: 'Before' if x < referendum_date else 'After')

In [47]:
# Optional: Save cleaned data
df.to_csv("merged_socialmedia_data_cleaned.csv", index=False)